In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,23782
2,Huelva,Confirmados PDIA 14 días,2998
3,Huelva,Tasa PDIA 14 días,"584,2118596176706"
4,Huelva,Confirmados PDIA 7 días,888
5,Huelva,Total Confirmados,24014
6,Huelva,Curados,12853
7,Huelva,Fallecidos,238


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  23782.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  6316.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 553 personas en los últimos 7 días 

Un positivo PCR cada 203 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,23782.0,888.0,2998.0,513170.0,173.042072,584.211860,106.0
Huelva-Costa,14477.0,585.0,1796.0,289548.0,202.039040,620.277122,83.0
Huelva (capital),6316.0,260.0,707.0,143837.0,180.760166,491.528605,45.0
Lepe,1924.0,81.0,264.0,27880.0,290.530846,946.915352,21.0
Condado-Campiña,6573.0,225.0,934.0,156231.0,144.017513,597.832696,15.0
Isla Cristina,1950.0,70.0,210.0,21393.0,327.209835,981.629505,8.0
Almonte,776.0,53.0,127.0,24507.0,216.264741,518.219284,5.0
Ayamonte,1164.0,63.0,198.0,21104.0,298.521607,938.210766,3.0
Villablanca,162.0,24.0,59.0,2885.0,831.889081,2045.060659,3.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villablanca,162.0,24.0,59.0,2885.0,831.889081,2045.060659,3.0
Berrocal,19.0,2.0,5.0,303.0,660.066007,1650.165017,0.0
Paymogo,72.0,3.0,16.0,1162.0,258.175559,1376.936317,0.0
Villalba del Alcor,436.0,4.0,38.0,3366.0,118.835413,1128.936423,0.0
Isla Cristina,1950.0,70.0,210.0,21393.0,327.209835,981.629505,8.0
Palma del Condado (La),959.0,19.0,106.0,10801.0,175.909638,981.390612,0.0
Lepe,1924.0,81.0,264.0,27880.0,290.530846,946.915352,21.0
Ayamonte,1164.0,63.0,198.0,21104.0,298.521607,938.210766,3.0
Bollullos Par del Condado,556.0,28.0,134.0,14387.0,194.620143,931.396400,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Escacena del Campo,128.0,1.0,11.0,2287.0,43.725404,480.979449,0.0,0.090909
San Bartolomé de la Torre,188.0,3.0,31.0,3761.0,79.766020,824.248870,0.0,0.096774
Gibraleón,499.0,5.0,51.0,12737.0,39.255712,400.408259,1.0,0.098039
Villalba del Alcor,436.0,4.0,38.0,3366.0,118.835413,1128.936423,0.0,0.105263
Lucena del Puerto,146.0,2.0,17.0,3261.0,61.330880,521.312481,0.0,0.117647
Calañas,169.0,2.0,14.0,2768.0,72.254335,505.780347,0.0,0.142857
Alájar,15.0,1.0,7.0,759.0,131.752306,922.266140,0.0,0.142857
San Juan del Puerto,350.0,10.0,59.0,9411.0,106.258634,626.925938,0.0,0.169492
Cartaya,863.0,20.0,117.0,20083.0,99.586715,582.582284,1.0,0.170940
